In [16]:
import os
import sys
sys.path.append('/Users/aghavamp/Desktop/Projects')
sys.path.append('/Users/aghavamp/Desktop/Projects/Functional_Fusion')
sys.path.append('/Users/aghavamp/Desktop/Projects/SUITPy')
import getpass
import importlib
import tqdm

import scipy.io as sio
import rsatoolbox as rsa
from rsatoolbox.io import spm as spm_io
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import surfAnalysisPy as surf
import SUITPy as suit
import nibabel as nb
import nitools as nt
from matplotlib.cm import ScalarMappable
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from pathlib import Path
import seaborn as sns
import PcmPy as pcm
import Functional_Fusion.atlas_map as am
import Functional_Fusion.reliability as rel
import glob
import matplotlib.patches as patches
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps

# SET PATHS:
baseDir = os.path.join('/Users', getpass.getuser(), 'Desktop', 'Projects', 'bimanual_wrist', 'data', 'fMRI')
bidsDir = 'BIDS'
anatomicalDir = 'anatomicals'
freesurferDir = 'surfaceFreesurfer'
surfacewbDir = 'surfaceWB' 
behavDir = 'behavioural'
regDir = 'ROI'
atlasDir = '/Volumes/diedrichsen_data$/data/Atlas_templates/fs_LR_32'



## R2 map

In [12]:
glm = 'single'
participants = [109,110,111,112,113,114,115]
participants = [113]
cmap = nilearn_cmaps['cold_white_hot']
tmax = 30

for _, sn in enumerate(participants):
    # Define the relevant gifti files for both left and right hemisphere:
    surf_white = [] # White -gray matter surface 
    surf_pial = []  # Pial surface
    surf_sulc = []  # Sulcal depth 
    hemN = ['L','R']
    for h,hem in enumerate(hemN):
        surf_white.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.white.32k.surf.gii'))
        surf_pial.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.pial.32k.surf.gii'))
        surf_sulc.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.sulc.32k.shape.gii'))
    
    spmT_names = sorted(glob.glob(os.path.join(baseDir, f'glm{glm}', f's{sn}', "R2.nii")))
    for spmT in spmT_names:
        # Map a Nifti to the surface of left hemisphere
        DL = surf.map.vol_to_surf([spmT],surf_pial[0], surf_white[0])
        # Map a Nifti to the surface of right hemisphere
        DR = surf.map.vol_to_surf([spmT],surf_pial[1],surf_white[1])

        D = {'L':DL, 'R':DR}

        Hem = ['L', 'R']
        region_names = ['?', 'S1', 'M1', 'PMd', 'PMv', 'SMA', 'V1', 'SPLa', '?']
        borders = {'L': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.L.border',
                   'R': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_R/fs_LR.32k.R.border'}
        
        # Combine data from both hemispheres
        all_data = np.concatenate([D['L'].flatten(), D['R'].flatten()])
        # Use percentiles
        # vmin = np.nanpercentile(all_data,20)
        vmax = tmax
        vmin = 0
        
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        for ax, H in zip(axs, Hem):
            plt.sca(ax)
            surf.plot.plotmap(D[H].mean(axis=1), f'fs32k_{H}',
                                underlay=None,
                                borders=borders[H],
                                cscale=[vmin, vmax],
                                cmap='jet',
                                underscale=[-1.5, 1],
                                alpha=.5,
                                new_figure=False,
                                colorbar=False,
                                # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
                                )
        # make colorbar
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        sm = ScalarMappable(norm=norm, cmap='jet')
        cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
        cbar.set_label('R-squared percentage')

        axs[0].set_title('Left hemisphere')
        axs[1].set_title('Right hemisphere')
        fig.suptitle(f'R2, s{sn}, glm{glm}, {spmT}')
        fig.tight_layout()
        plt.savefig(f'../figures/glmsingle/tmaps/R2_s{sn}.pdf', bbox_inches="tight")
        plt.close()
        
    # D = {'L':DL_avg, 'R':DR_avg}
    # all_data = np.concatenate([DL_avg.flatten(), DR_avg.flatten()])
    # # Use percentiles
    # # vmin = np.nanpercentile(all_data,20)
    # vmax = np.nanpercentile(all_data, tmax)
    # vmin = -vmax
    # fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    # for ax, H in zip(axs, Hem):
    #     plt.sca(ax)
    #     surf.plot.plotmap(D[H].mean(axis=1), f'fs32k_{H}',
    #                         underlay=None,
    #                         borders=borders[H],
    #                         cscale=[vmin, vmax],
    #                         cmap=cmap,
    #                         underscale=[-1.5, 1],
    #                         alpha=.5,
    #                         new_figure=False,
    #                         colorbar=False,
    #                         # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
    #                         )
    # # make colorbar
    # norm = plt.Normalize(vmin=vmin, vmax=vmax)
    # sm = ScalarMappable(norm=norm, cmap=cmap)
    # cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
    # cbar.set_label('t-value')

    # axs[0].set_title('Left hemisphere')
    # axs[1].set_title('Right hemisphere')
    # fig.suptitle(f'avg, glm{glm}, {spmT.split("/")[-1].split(".")[0]}')
    # fig.tight_layout()
    # plt.savefig(f'../figures/glmsingle/tmaps/avg_glm{glm}_{spmT.split("/")[-1].split(".")[0]}.pdf', bbox_inches="tight")



/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_74181/2599161843.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


## per condition/subject tmap

In [17]:
glm = 'single'
participants = [109,110,111,112,113,114,115]
participants = [113]
cmap = 'bwr'
threshold = 1.5
tmax = 99.9

for _, sn in enumerate(participants):
    # Define the relevant gifti files for both left and right hemisphere:
    surf_white = [] # White -gray matter surface 
    surf_pial = []  # Pial surface
    surf_sulc = []  # Sulcal depth 
    hemN = ['L','R']
    for h,hem in enumerate(hemN):
        surf_white.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.white.32k.surf.gii'))
        surf_pial.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.pial.32k.surf.gii'))
        surf_sulc.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.sulc.32k.shape.gii'))
    
    spmT_names = sorted(glob.glob(os.path.join(baseDir, f'glm{glm}', f's{sn}', "tmap_*.nii")))
    for spmT in spmT_names:
        # Map a Nifti to the surface of left hemisphere
        DL = surf.map.vol_to_surf([spmT],surf_pial[0], surf_white[0])
        DL[np.abs(DL) < threshold] = 0
        # Map a Nifti to the surface of right hemisphere
        DR = surf.map.vol_to_surf([spmT],surf_pial[1],surf_white[1])
        DR[np.abs(DR) < threshold] = 0
        
        # DL_avg += DL/len(participants)
        # DR_avg += DR/len(participants)

        D = {'L':DL, 'R':DR}

        Hem = ['L', 'R']
        region_names = ['?', 'S1', 'M1', 'PMd', 'PMv', 'SMA', 'V1', 'SPLa', '?']
        borders = {'L': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.L.border',
                   'R': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_R/fs_LR.32k.R.border'}
        
        # Combine data from both hemispheres
        all_data = np.concatenate([D['L'].flatten(), D['R'].flatten()])
        # Use percentiles
        # vmin = np.nanpercentile(all_data,20)
        vmax = np.nanpercentile(all_data, tmax)
        vmin = -vmax
        
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        for ax, H in zip(axs, Hem):
            plt.sca(ax)
            surf.plot.plotmap(D[H].mean(axis=1), f'fs32k_{H}',
                                underlay=None,
                                borders=borders[H],
                                cscale=[vmin, vmax],
                                cmap='jet',
                                underscale=[-1.5, 1],
                                alpha=.5,
                                new_figure=False,
                                colorbar=False,
                                # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
                                )
        # make colorbar
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        sm = ScalarMappable(norm=norm, cmap='jet')
        cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
        cbar.set_label('t-value')

        axs[0].set_title('Left hemisphere')
        axs[1].set_title('Right hemisphere')
        fig.suptitle(f'avg t-maps, s{sn}, glm{glm}, {spmT}')
        fig.tight_layout()
        filename = name = os.path.splitext(os.path.basename(spmT))[0]
        filename = filename.split('tmap_', 1)[1]
        plt.savefig(f'../figures/glmsingle/tmaps/{filename}_s{sn}.pdf', bbox_inches="tight")
        plt.close()
        
    # D = {'L':DL_avg, 'R':DR_avg}
    # all_data = np.concatenate([DL_avg.flatten(), DR_avg.flatten()])
    # # Use percentiles
    # # vmin = np.nanpercentile(all_data,20)
    # vmax = np.nanpercentile(all_data, tmax)
    # vmin = -vmax
    # fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    # for ax, H in zip(axs, Hem):
    #     plt.sca(ax)
    #     surf.plot.plotmap(D[H].mean(axis=1), f'fs32k_{H}',
    #                         underlay=None,
    #                         borders=borders[H],
    #                         cscale=[vmin, vmax],
    #                         cmap=cmap,
    #                         underscale=[-1.5, 1],
    #                         alpha=.5,
    #                         new_figure=False,
    #                         colorbar=False,
    #                         # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
    #                         )
    # # make colorbar
    # norm = plt.Normalize(vmin=vmin, vmax=vmax)
    # sm = ScalarMappable(norm=norm, cmap=cmap)
    # cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
    # cbar.set_label('t-value')

    # axs[0].set_title('Left hemisphere')
    # axs[1].set_title('Right hemisphere')
    # fig.suptitle(f'avg, glm{glm}, {spmT.split("/")[-1].split(".")[0]}')
    # fig.tight_layout()
    # plt.savefig(f'../figures/glmsingle/tmaps/avg_glm{glm}_{spmT.split("/")[-1].split(".")[0]}.pdf', bbox_inches="tight")




KeyboardInterrupt: 

## mask.nii on surface

In [15]:
glm = 'single'
participants = [101,102,103,104,106,107,108,109,110,111,112,113,114,115]
# participants = [113]
cmap = nilearn_cmaps['cold_white_hot']
tmax = 1

for _, sn in enumerate(participants):
    # Define the relevant gifti files for both left and right hemisphere:
    surf_white = [] # White -gray matter surface 
    surf_pial = []  # Pial surface
    surf_sulc = []  # Sulcal depth 
    hemN = ['L','R']
    for h,hem in enumerate(hemN):
        surf_white.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.white.32k.surf.gii'))
        surf_pial.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.pial.32k.surf.gii'))
        surf_sulc.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.sulc.32k.shape.gii'))
    
    spmT_names = sorted(glob.glob(os.path.join(baseDir, f'glm{glm}', f's{sn}', "mask.nii")))
    for spmT in spmT_names:
        # Map a Nifti to the surface of left hemisphere
        DL = surf.map.vol_to_surf([spmT],surf_pial[0], surf_white[0])
        # Map a Nifti to the surface of right hemisphere
        DR = surf.map.vol_to_surf([spmT],surf_pial[1],surf_white[1])

        D = {'L':DL, 'R':DR}

        Hem = ['L', 'R']
        region_names = ['?', 'S1', 'M1', 'PMd', 'PMv', 'SMA', 'V1', 'SPLa', '?']
        borders = {'L': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.L.border',
                   'R': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_R/fs_LR.32k.R.border'}
        
        # Combine data from both hemispheres
        all_data = np.concatenate([D['L'].flatten(), D['R'].flatten()])
        # Use percentiles
        # vmin = np.nanpercentile(all_data,20)
        vmax = tmax
        vmin = 0
        
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        for ax, H in zip(axs, Hem):
            plt.sca(ax)
            surf.plot.plotmap(D[H].mean(axis=1), f'fs32k_{H}',
                                underlay=None,
                                borders=borders[H],
                                cscale=[vmin, vmax],
                                cmap='jet',
                                underscale=[-1.5, 1],
                                alpha=.5,
                                new_figure=False,
                                colorbar=False,
                                # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
                                )
        # make colorbar
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        sm = ScalarMappable(norm=norm, cmap='jet')
        cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
        cbar.set_label('R-squared percentage')
        
        axs[0].set_title('Left hemisphere')
        axs[1].set_title('Right hemisphere')
        fig.suptitle(f'glm mask, s{sn}, glm{glm}, {spmT}')
        fig.tight_layout()
        plt.savefig(f'../figures/glmsingle/tmaps/glmmask_s{sn}.pdf', bbox_inches="tight")
        plt.close()
  


/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_74181/808450440.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_74181/808450440.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_74181/808450440.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_74181/808450440.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_74181/808450440.py:62: UserWarning: This figure includes Axes that are not co